refer to this for installation https://github.com/abetlen/llama-cpp-python#installation-with-openblas--cublas--clblast--metal 

In [37]:
# Uncomment to install dependencies
%pip install llama_index
%pip install llama-cpp-python
%pip install transformers
%pip install torch
%pip install pypdf
%pip install tqdm


Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [38]:
import logging
import sys

from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

logging.basicConfig(stream=sys.stdout, level=logging.INFO)  # Change INFO to DEBUG if you want more extensive logging
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

llm = LlamaCPP(
    model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q5_K_M.gguf",
    
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    
    temperature=0.0,
    max_new_tokens=1024,
    
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,  # note, this sets n_ctx in the model_kwargs below, so you don't need to pass it there.
    
    # kwargs to pass to __call__()
    generate_kwargs={},
    
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 30}, # I need to play with this and see if it actually helps
    
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [39]:
# Create an index of your documents
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

storage_directory = "./storage"

documents = SimpleDirectoryReader('./testdata').load_data()

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)


index = VectorStoreIndex.from_documents(documents, service_context=service_context)
# Persist the index to disk
display(index)
index.storage_context.persist(persist_dir=storage_directory)

**********
Trace: index_construction
    |_node_parsing ->  0.101956 seconds
      |_chunking ->  0.003991 seconds
      |_chunking ->  0.006986 seconds
      |_chunking ->  0.00199 seconds
      |_chunking ->  0.001999 seconds
      |_chunking ->  0.000994 seconds
      |_chunking ->  0.000997 seconds
      |_chunking ->  0.000993 seconds
      |_chunking ->  0.001999 seconds
      |_chunking ->  0.000997 seconds
      |_chunking ->  0.001994 seconds
      |_chunking ->  0.000995 seconds
      |_chunking ->  0.001001 seconds
      |_chunking ->  0.000994 seconds
      |_chunking ->  0.001 seconds
      |_chunking ->  0.005984 seconds
      |_chunking ->  0.005981 seconds
      |_chunking ->  0.004989 seconds
      |_chunking ->  0.004987 seconds
      |_chunking ->  0.006978 seconds
      |_chunking ->  0.005987 seconds
      |_chunking ->  0.000994 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.000997 seconds
      |_chunking ->  0.001001 seconds
      |_chunking -> 

In [40]:
# Now you can load the index from disk when needed, and not rebuild it each time.
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index import StorageContext, load_index_from_storage

# transcript_directory = "transcripts/ancient-aliens-official"
storage_directory = "./storage"

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)

storage_context = StorageContext.from_defaults(persist_dir=storage_directory)
index = load_index_from_storage(storage_context, service_context=service_context)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
**********
Trace: index_construction
**********


In [41]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

query = "Given the quiz from quiz.txt and the results from choices.txt describe my personality and the archetype from the information in all the files provided. \n" \
        "Please use the following format: \n" \
        "Your personality is <personality> because <reasoning>. \n" \
        "Your archetype is <archetype> because <reasoning>. \n" \
        "Personality should not be aan archetype."

        # "Please summarize your reasoning into 3 sentences. \n" \


response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

**********
Trace: query
    |_query ->  317.511931 seconds
      |_retrieve ->  0.030923 seconds
        |_embedding ->  0.021941 seconds
      |_synthesize ->  317.481008 seconds
        |_templating ->  0.0 seconds
        |_llm ->  317.458696 seconds
**********


<b>  Based on your quiz results and choices, your personality is best described as a "Community Mentor" because you prioritize helping others and contributing to the community. Your archetype is the "Kampong Leader" because you value community bonding and mentorship, and you are willing to invest time and resources to support those in need.

Your results indicate that you resonate with the values of a traditional Singaporean community leader, who prioritizes the well-being of others and works towards the betterment of the community as a whole. You are likely someone who is approachable, empathetic, and willing to listen to others, and you may have a strong sense of responsibility towards your community.

In terms of your personality, you seem to be a caring and compassionate individual who values relationships and community over material possessions or personal gain. You are likely someone who is patient, understanding, and willing to lend a helping hand when needed. Your archetype reflects these qualities, as the Kampong Leader is known for their wisdom, kindness, and dedication to their community.</b>

In [42]:
raise Exception("Stop here")

Exception: Stop here

In [ ]:
query = "Would you rather be a Samsui women or live in such a stressful society in Singapore's context \n" \
        "Please summarize your reasoning into 2 sentences."

response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit


**********
Trace: query
    |_query ->  474.15284 seconds
      |_retrieve ->  0.553662 seconds
        |_embedding ->  0.492307 seconds
      |_synthesize ->  473.596518 seconds
        |_templating ->  0.0 seconds
        |_llm ->  473.51626 seconds
**********


<b>  Based on the given context, I would rather be a Samsui woman than live in a stressful society in Singapore's context. As a Samsui woman, I would have a strong support system and a sense of community, while also having the opportunity to contribute to the development of Singapore through my hard work and resilience. In contrast, living in a stressful society in Singapore's context may lead to burnout and a lack of fulfillment, despite the potential for higher wages and better living conditions.</b>

In [ ]:
query = "Given the quiz from quiz.txt and the choices from choices.txt describe my personality and which archetype I'm most probably will be. \n" \
        "Please summarize your reasoning into 2 sentences."

response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit


**********
Trace: query
    |_query ->  38.681917 seconds
      |_retrieve ->  0.184877 seconds
        |_embedding ->  0.165846 seconds
      |_synthesize ->  38.49704 seconds
        |_templating ->  0.0 seconds
        |_llm ->  38.479332 seconds
**********


<b>  Based on your responses to the quiz questions, you seem to possess a combination of traits from different archetypes. Your desire to contribute to the community and help others (Question 1), your preference for walking or eco-friendly transportation (Question 2), and your interest in cultural events and education (Question 3) suggest that you resonate with the Samsui Woman archetype. Additionally, your willingness to volunteer and teach (Question 4) and your focus on community bonding and mentorship (Question 5) further support this conclusion.</b>